In [509]:
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import numpy as np
from numpy import NaN, NAN,nan
import statistics

all_data={}
for column in ['AAL','ALK','AVH','CEA','ZNH','VLRS','CPA','DAL','GOL','LTM','UAL']:
    all_data[column]=yf.download(column, start = '2014-06-02', end='2016-06-13')

luvdf=pd.read_csv('LUV.csv')
luvdf['Date'] = pd.to_datetime(luvdf['Date'])
luvdf.index = luvdf['Date']
luvdf = luvdf.drop(columns = ['Date'])
luvdf = luvdf.iloc[21:]
luvdf=luvdf.iloc[:-1]

dict1={ 'Date':np.array([k for k in all_data['AAL'].index])}
for column in ['AAL','ALK','AVH','CEA','ZNH','VLRS','CPA','DAL','GOL','LTM','UAL']:
    dict1[column]=np.array([k for k in all_data[column]['Adj Close']])
price_industry = pd.DataFrame(dict1) 
price_industry.index = price_industry['Date']
price_industry = price_industry.drop(columns = ['Date'])

price_return=price_industry.pct_change(periods = 1)
price_return=price_return.replace(to_replace=[NaN],value=0)
lag=pd.DataFrame({'price_LUV':np.array([k for k in luvdf['Adj Close']]),
                  'daily_return_airline_industry' :price_return.mean(axis=1),
                'MovingAverage(5)':luvdf['Adj Close'].rolling(5).mean(),
                  'MovingAverage(25)':luvdf['Adj Close'].rolling(25).mean()
               })
lag


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,price_LUV,daily_return_airline_industry,MovingAverage(5),MovingAverage(25)
Date,,,,
2014-06-02,14.711601,0.000000,NaN,NaN
2014-06-03,14.475902,0.000874,NaN,NaN
2014-06-04,14.259844,-0.005360,NaN,NaN
2014-06-05,14.289306,0.002586,NaN,NaN
2014-06-06,14.338411,0.014908,14.415013,NaN
...,...,...,...,...
2016-06-06,2.160000,0.020655,2.098000,2.2484
2016-06-07,2.170000,0.016486,2.128000,2.2488
2016-06-08,2.160000,0.008766,2.128000,2.2464


# HYPERPARAMETER_OPTIMIZATION

In [510]:
price=np.array(luvdf['Adj Close'])
maximum=0
ma_small=0
ma_big=0


for j in range(10,35,1):
    MA_BIG=np.array(luvdf['Adj Close'].rolling(j).mean())

    for i in range(2,j//2+1):
        MA_SMALL=np.array(luvdf['Adj Close'].rolling(i).mean())

        MA_small_industry=np.array(price_industry.mean(axis=1).rolling(i).mean())
        MA_big_industry=np.array(price_industry.mean(axis=1).rolling(j).mean())
        
        #INDUSTRY SIGNAL
        signals_industry=[0]
        for m in range(1,MA_small_industry.shape[0]-1):
            if MA_small_industry[m-1]==nan or MA_big_industry[m-1]==nan:
                signals_industry.append(0)
                continue
            if MA_small_industry[m-1]>=MA_big_industry[m-1] and MA_small_industry[m-2]<MA_big_industry[m-2]:
                signals_industry.append(1)
                continue
            elif MA_small_industry[m-1]<=MA_big_industry[m-1] and MA_small_industry[m-2]>MA_big_industry[m-2]:
                signals_industry.append(-1)
                continue
            else:
                signals_industry.append(0)
        
        signals=[0] 
        for k in range(1,MA_BIG.shape[0]-1):
            if MA_SMALL[k-1]==nan or MA_BIG[k-1]==nan:
                signals.append(0)
                continue
            elif (MA_SMALL[k-1]>=MA_BIG[k-1] and MA_SMALL[k-2]<MA_BIG[k-2]):
                flag1=False
                for n in range(k-1,k-9,-1):
                    if(signals_industry[n]==1):
                        signals.append(1)
                        flag1=True
                        break
                if(flag1==False):               
                    signals.append(0)


            elif (MA_SMALL[k-1]<=MA_BIG[k-1] and MA_SMALL[k-2]>MA_BIG[k-2] ):
                flag2=False
                for n in range(k-2,k-9,-1):
                    if(signals_industry[n]==-1):
                        signals.append(-1)
                        flag2=True
                        break
                if(flag2==False):               
                    signals.append(0)
            else:
                signals.append(0)
        
        signals.append(0)#last day closing all the positions so no signal generated
        signals.append(sum(signals)-0)#adding terminal wealth
        signal=np.array(signals)  
        Pnl=[-price[l] if signal[l]== 1 else +price[l] if signal[l] ==-1 else 0 for l in  range(0,len(signal)-1)]
        sumPnl=  sum(Pnl)+signal[-1]*(price[-1] )
        print(i,j,sumPnl)
        if(sumPnl>maximum):
            optimized_Pnl=Pnl
            optimized_signal=signal
            maximum=sumPnl
            ma_small=i
            ma_big=j
sumPnl=maximum
print(optimized_signal,ma_small,ma_big,maximum)


[[ 0.00000000e+00]
 [-3.08096489e-02]
 [-2.21168063e-01]
 [ 3.10077519e-01]
 [-4.33522521e-02]
 [ 5.31683918e-01]
 [-3.67397396e-01]
 [-2.67717561e-01]
 [ 4.08824053e-01]
 [-2.40181841e-01]
 [ 1.00299152e+00]
 [-3.75539330e-01]
 [ 6.52404996e-02]
 [-4.12498441e-01]
 [ 8.60084926e-01]
 [ 4.79853898e-01]
 [-4.00000000e-01]
 [-2.68414963e-01]
 [-4.97276401e-02]
 [-3.31360947e-01]
 [ 3.87444690e-01]
 [ 7.60813235e-01]
 [-2.92393027e-01]
 [ 6.41657335e-01]
 [-3.50906256e-01]
 [-3.39138268e-01]
 [ 5.72012721e-01]
 [-2.61849711e-01]
 [-9.00548160e-03]
 [-3.60529435e-01]
 [-6.35464937e-01]
 [ 2.16779661e+00]
 [ 3.37078652e-02]
 [ 5.02070393e-02]
 [ 2.76490882e-01]
 [ 4.55791506e-01]
 [-3.58042700e-03]
 [-9.34256055e-02]
 [-4.42894891e-01]
 [ 1.43056653e+00]
 [-1.65980052e-01]
 [-6.16404524e-01]
 [ 4.64927143e-01]
 [ 5.55170021e-02]
 [-2.49616480e-01]
 [ 1.06016355e-01]
 [-7.63137048e-02]
 [-3.94511149e-02]
 [ 1.83630952e-01]
 [ 3.62836309e-01]
 [ 4.14391144e-01]
 [-2.71067049e-01]
 [ 3.9209019

2 14 -3.678700999999999
3 14 9.358564
4 14 -34.29946100000001
5 14 -19.699301000000006
6 14 -8.638806
7 14 21.900860999999995
2 15 2.0812989999999987
3 15 8.518563999999996
4 15 -13.436717000000002
5 15 -31.093479000000002
6 15 -4.673111999999996
7 15 -13.879553000000014
2 16 9.818779999999997
3 16 9.858564
4 16 -6.884645
5 16 -16.399301
6 16 -15.922665000000004
7 16 -16.48930200000001
8 16 0.42999899999999736
2 17 5.168779999999998
3 17 14.207130999999997
4 17 -1.7346450000000018
5 17 -3.9831120000000007
6 17 -1.413111999999999
7 17 -12.319302
8 17 0.35892399999999824
2 18 -6.604941000000002
3 18 10.237131000000002
4 18 -14.936717000000005
5 18 0.1568880000000008
6 18 0.026888000000001355
7 18 -12.389302
8 18 -0.941076000000002
9 18 0.08344499999999977
2 19 -1.9692360000000044
3 19 -3.461819999999999
4 19 -3.640528000000003
5 19 3.816888000000003
6 19 4.246887999999999
7 19 -7.400377000000001
8 19 3.9029069999999995
9 19 0.08344499999999977
2 20 -15.072134000000013
3 20 -7.46181999999

In [511]:
lag['SIGNAL']=np.array(optimized_signal[:-1])
lag['P/L']=np.array(optimized_Pnl)
lag

,price_LUV,daily_return_airline_industry,MovingAverage(5),MovingAverage(25),SIGNAL,P/L
Date,,,,,,
2014-06-02,14.711601,0.000000,NaN,NaN,0,0.0
2014-06-03,14.475902,0.000874,NaN,NaN,0,0.0
2014-06-04,14.259844,-0.005360,NaN,NaN,0,0.0
2014-06-05,14.289306,0.002586,NaN,NaN,0,0.0
2014-06-06,14.338411,0.014908,14.415013,NaN,0,0.0
...,...,...,...,...,...,...
2016-06-06,2.160000,0.020655,2.098000,2.2484,0,0.0
2016-06-07,2.170000,0.016486,2.128000,2.2488,0,0.0
2016-06-08,2.160000,0.008766,2.128000,2.2464,0,0.0


In [512]:
days_when_received_signal=[]
for i in range(len(optimized_signal)):
    if optimized_signal[i]!=0:
        days_when_received_signal.append(i)
print(days_when_received_signal)

[68, 77, 150, 194, 218, 236, 263, 282, 325, 343, 351, 392, 427, 478, 486, 512]


# volatility 

In [513]:
def volatility_calculator(optimized_signal,sumPnl):
    adj_close_luv=price
    adj_close_return=np.diff(adj_close_luv)
    vol=[]
    buy=False
    sell=False
    for i in range(len(adj_close_return)):
        if optimized_signal[i]==1:
            if sell==True:
                buy=False
                sell=False
                vol.append(adj_close_return[i])
            else:
                buy=True
                sell=False
        if optimized_signal[i]==-1:
            if buy==True:
                vol.append(adj_close_return[i])
                buy=False
                sell=False
            else:
                buy=False
                sell=True
        if buy==False and sell==False:
            continue
        if buy==True and sell==False:
            vol.append(adj_close_return[i])
        if buy==False and sell==True:
            vol.append(adj_close_return[i])
            
    if optimized_signal[-1]!=0:
        vol.append(adj_close_return[-1])

    print("mean return=",sumPnl/(len(days_when_received_signal)-1),"volatility=",np.array(vol).std(),"sharperatio=",(sumPnl/(len(days_when_received_signal)-1))/np.array(vol).std())


In [514]:
volatility_calculator(optimized_signal,sumPnl)

mean return= 1.4600573999999997 volatility= 0.328048949076361 sharperatio= 4.450730307507059


## is my strategy better? following are the results of using simple crossover strategy for LUV stock witout considering lag 

In [515]:
MA_small=np.array(luvdf['Adj Close'].rolling(8).mean())
MA_large=np.array(luvdf['Adj Close'].rolling(20).mean())
signal=[]
for i in range(1,MA_small.shape[0]-1):
    if MA_small[i-1]==nan or MA_large[i-1]==nan:
        signal.append(0)
        continue
    if MA_small[i-1]>=MA_large[i] and MA_small[i-2]<MA_large[i-2]:
        signal.append(1)
        continue
    elif MA_small[i-1]<=MA_large[i-1] and MA_small[i-2]>MA_large[i-2]:
        signal.append(-1)
        continue
    else:
        signal.append(0)
        
signal.append(0)     
signal.append(sum(signal)-0)
signal=np.array(signal)  
Pnl=[-price[l] if signal[l]== 1 else +price[l] if signal[l] ==-1 else 0 for l in  range(0,len(signal)-1)]
sumPnl= sum(Pnl)+signal[-1]*(price[-1])
print(signal,sumPnl)
days_when_received_signal=[]
for i in range(len(signal)):
    if signal[i]!=0:
        days_when_received_signal.append(i)
print(days_when_received_signal)
volatility_calculator(signal,sumPnl)


[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  1  1  0 -1  0  0  0  0  0  0  0  0
  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0
  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0

## TRADING SYSYTEM

## BACKTESTING

In [525]:
import datetime
import pandas as pd
import numpy as np

class trading_strategy:


    def __init__(self):

        self.todays_date = datetime.datetime.now().date()
        self.index = pd.date_range(self.todays_date, periods=1)
        self.columns = ['AdjClose']
        self.df= pd.DataFrame(index=self.index, columns=self.columns)
        self.df=self.df.drop(self.df.index[0])
        self.moving_average_10=[]
        self.moving_average_3=[]
        self.pnl=0
    
    def process_tick(self, adujusted_price):
        self.todays_date+=datetime.timedelta(days=1)
        row = pd.Series({'AdjClose':adujusted_price},name=self.todays_date)
        self.df = self.df.append(row)
        self.moving_average_3=np.array(self.df['AdjClose'].rolling(3).mean())
        self.moving_average_10=np.array(self.df['AdjClose'].rolling(10).mean())
        
        signal_received=self.check_signal()
        if(signal_received==1):
            self.generate_buy_order(adujusted_price)
        
        elif(signal_received==-1):
            self.generate_sell_order(adujusted_price)
            
        self.update_pnl(signal_received,adujusted_price)
   
            
    def generate_buy_order(self,adjusted_price):
        print("BUY",adjusted_price,self.todays_date)
    def generate_sell_order(self,adjusted_price):
        print("SELL",adjusted_price,self.todays_date)    
    def check_signal(self):
        current_tick=len(self.moving_average_3)-1
        if self.moving_average_3[current_tick]==nan or self.moving_average_10[current_tick]==nan:
            return 0
        elif self.moving_average_3[current_tick]>=self.moving_average_10[current_tick] and self.moving_average_3[current_tick-1]<self.moving_average_10[current_tick-1]:
            return 1
        elif self.moving_average_3[current_tick]<=self.moving_average_10[current_tick] and self.moving_average_3[current_tick-1]>self.moving_average_10[current_tick-1]:
            return -1
        else:
            return 0     
    def update_pnl(self,signal_received,adujusted_price):
        if(signal_received==1):
            self.pnl+=adujusted_price
        elif(signal_received==-1):
            self.pnl-=adujusted_price
        else:
            self.pnl+=0
    def display_pnl(self):
        print("pnl=",self.pnl)

        
ts_new=trading_strategy()
list1=luvdf['Adj Close'].values
for i in list1:
    ts_new.process_tick(i)

ts_new.display_pnl()

BUY 58.35857009887695 2020-06-04
SELL 60.09122848510742 2020-06-21
BUY 51.296024322509766 2020-08-01
SELL 55.45831298828125 2020-09-03
BUY 53.62458038330078 2020-09-19
SELL 63.63039779663086 2020-10-18
BUY 63.649845123291016 2020-10-28
SELL 60.480464935302734 2020-11-03
BUY 56.56250762939453 2020-11-19
SELL 56.3972282409668 2020-11-28
BUY 56.752071380615234 2020-12-01
SELL 56.86898422241211 2020-12-03
BUY 58.16478729248047 2020-12-04
SELL 56.294158935546875 2020-12-12
BUY 54.384559631347656 2020-12-31
SELL 52.494449615478516 2021-01-04
BUY 51.64681625366211 2021-01-20
SELL 49.5423698425293 2021-01-24
BUY 50.16590881347656 2021-01-28
SELL 49.766448974609375 2021-01-30
BUY 50.184967041015625 2021-02-08
SELL 50.00905227661133 2021-02-17
BUY 50.68339920043945 2021-02-23
SELL 50.73226547241211 2021-03-08
BUY 55.41358947753906 2021-03-10
SELL 60.408111572265625 2021-04-21
BUY 56.792442321777344 2021-05-08
SELL 56.70425796508789 2021-05-10
BUY 50.38418197631836 2021-05-20
SELL 50.531162261962

# BACKTESTING

In [527]:

luvdf=yf.download("LUV", start = '2017-06-02', end='2019-06-13')
ts_new2=trading_strategy()
list1=luvdf['Adj Close'].values
for i in list1:
    ts_new2.process_tick(i)

ts_new2.display_pnl()

[*********************100%***********************]  1 of 1 completed
BUY 58.35857009887695 2020-06-04
SELL 60.09122848510742 2020-06-21
BUY 51.296024322509766 2020-08-01
SELL 55.45831298828125 2020-09-03
BUY 53.62458038330078 2020-09-19
SELL 63.63039779663086 2020-10-18
BUY 63.649845123291016 2020-10-28
SELL 60.480464935302734 2020-11-03
BUY 56.56250762939453 2020-11-19
SELL 56.3972282409668 2020-11-28
BUY 56.752071380615234 2020-12-01
SELL 56.86898422241211 2020-12-03
BUY 58.16478729248047 2020-12-04
SELL 56.294158935546875 2020-12-12
BUY 54.384559631347656 2020-12-31
SELL 52.494449615478516 2021-01-04
BUY 51.64681625366211 2021-01-20
SELL 49.5423698425293 2021-01-24
BUY 50.16590881347656 2021-01-28
SELL 49.766448974609375 2021-01-30
BUY 50.184967041015625 2021-02-08
SELL 50.00905227661133 2021-02-17
BUY 50.68339920043945 2021-02-23
SELL 50.73226547241211 2021-03-08
BUY 55.41358947753906 2021-03-10
SELL 60.408111572265625 2021-04-21
BUY 56.792442321777344 2021-05-08
SELL 56.7042579650